# Chanterelle Habitat Analysis - Exploration Notebook

This notebook provides tools for:
1. Visualizing intermediate and final layers
2. Validating the suitability model
3. Comparing against known chanterelle locations

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import rasterio
from rasterio.plot import show
from pathlib import Path

%matplotlib inline
plt.style.use('dark_background')

In [ ]:
# Define paths
PROJECT_ROOT = Path('..').resolve()
PROCESSED_DIR = PROJECT_ROOT / 'data' / 'processed'
OUTPUT_DIR = PROJECT_ROOT / 'data' / 'output'

print(f"Project root: {PROJECT_ROOT}")
print(f"Processed data: {PROCESSED_DIR}")
print(f"Output data: {OUTPUT_DIR}")

## 1. Create Custom Color Maps

In [ ]:
# Chanterelle-themed colormap (purple to gold)
chanterelle_colors = [
    (0.0, '#1a0033'),   # Dark purple
    (0.2, '#4b0082'),   # Indigo
    (0.4, '#8a2be2'),   # Blue-violet
    (0.5, '#ff6600'),   # Orange
    (0.7, '#ffc800'),   # Gold
    (1.0, '#ffeb64'),   # Bright gold
]

# Create colormap
positions = [c[0] for c in chanterelle_colors]
colors = [c[1] for c in chanterelle_colors]
chanterelle_cmap = LinearSegmentedColormap.from_list('chanterelle', list(zip(positions, colors)))

# Access colormap (gray to green)
access_colors = ['#1a1a1a', '#404040', '#808080', '#90c090', '#40ff40']
access_cmap = LinearSegmentedColormap.from_list('access', access_colors)

# Display colormaps
fig, axes = plt.subplots(1, 2, figsize=(12, 2))
gradient = np.linspace(0, 1, 256).reshape(1, -1)

axes[0].imshow(gradient, aspect='auto', cmap=chanterelle_cmap)
axes[0].set_title('Habitat Suitability')
axes[0].set_yticks([])

axes[1].imshow(gradient, aspect='auto', cmap=access_cmap)
axes[1].set_title('Access Quality')
axes[1].set_yticks([])

plt.tight_layout()
plt.show()

## 2. Load and Visualize Layers

In [ ]:
def load_raster(filepath):
    """Load a raster and return data with mask."""
    with rasterio.open(filepath) as src:
        data = src.read(1)
        nodata = src.nodata
        bounds = src.bounds
        
    # Create masked array
    if nodata is not None:
        data = np.ma.masked_equal(data, nodata)
    data = np.ma.masked_invalid(data)
    
    return data, bounds

def plot_layer(filepath, title, cmap='viridis', vmin=None, vmax=None, figsize=(12, 8)):
    """Plot a raster layer."""
    data, bounds = load_raster(filepath)
    
    fig, ax = plt.subplots(figsize=figsize)
    
    extent = [bounds.left, bounds.right, bounds.bottom, bounds.top]
    im = ax.imshow(data, cmap=cmap, extent=extent, vmin=vmin, vmax=vmax)
    
    ax.set_title(title, fontsize=14)
    ax.set_xlabel('Easting (m)')
    ax.set_ylabel('Northing (m)')
    
    cbar = plt.colorbar(im, ax=ax, shrink=0.8)
    
    # Print statistics
    valid_data = data.compressed()
    print(f"\n{title} Statistics:")
    print(f"  Min: {valid_data.min():.3f}")
    print(f"  Max: {valid_data.max():.3f}")
    print(f"  Mean: {valid_data.mean():.3f}")
    print(f"  Std: {valid_data.std():.3f}")
    
    plt.tight_layout()
    return fig, ax

In [ ]:
# List available processed layers
print("Processed layers:")
for f in sorted(PROCESSED_DIR.glob('*.tif')):
    print(f"  - {f.name}")

print("\nOutput layers:")
for f in sorted(OUTPUT_DIR.glob('*.tif')):
    print(f"  - {f.name}")

In [ ]:
# Visualize elevation
elev_path = PROCESSED_DIR / 'elevation_ft.tif'
if elev_path.exists():
    plot_layer(elev_path, 'Elevation (feet)', cmap='terrain')

In [ ]:
# Visualize habitat suitability
habitat_path = OUTPUT_DIR / 'habitat_suitability.tif'
if habitat_path.exists():
    plot_layer(habitat_path, 'Habitat Suitability', cmap=chanterelle_cmap, vmin=0, vmax=1)

In [ ]:
# Visualize access quality
access_path = OUTPUT_DIR / 'access_quality.tif'
if access_path.exists():
    plot_layer(access_path, 'Access Quality', cmap=access_cmap, vmin=0, vmax=1)

## 3. Score Distribution Analysis

In [ ]:
# Histogram of habitat scores
if habitat_path.exists():
    data, _ = load_raster(habitat_path)
    valid_data = data.compressed()
    
    fig, ax = plt.subplots(figsize=(10, 6))
    
    n, bins, patches = ax.hist(valid_data, bins=50, edgecolor='black', alpha=0.7)
    
    # Color bars by score
    for i, (patch, bin_val) in enumerate(zip(patches, bins[:-1])):
        color = chanterelle_cmap(bin_val)
        patch.set_facecolor(color)
    
    ax.set_xlabel('Suitability Score')
    ax.set_ylabel('Cell Count')
    ax.set_title('Distribution of Habitat Suitability Scores')
    
    # Add vertical lines for thresholds
    for thresh in [0.4, 0.6, 0.8]:
        ax.axvline(x=thresh, color='white', linestyle='--', alpha=0.7)
    
    plt.tight_layout()
    plt.show()
    
    # Calculate percentages
    print("\nScore Distribution:")
    for threshold in [0.2, 0.4, 0.6, 0.8]:
        pct = np.sum(valid_data >= threshold) / len(valid_data) * 100
        print(f"  >= {threshold}: {pct:.1f}% of area")

## 4. Multi-Layer Comparison

In [ ]:
# Compare all input layers in a grid
layers_to_plot = [
    ('elevation_ft.tif', 'Elevation (ft)', 'terrain'),
    ('slope_degrees.tif', 'Slope (°)', 'Reds'),
    ('aspect_degrees.tif', 'Aspect (°)', 'hsv'),
    ('canopy_cover.tif', 'Canopy Cover (%)', 'Greens'),
    ('soil_ph.tif', 'Soil pH', 'RdYlBu_r'),
    ('annual_precip_inches.tif', 'Precip (in)', 'Blues'),
]

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for ax, (filename, title, cmap) in zip(axes, layers_to_plot):
    filepath = PROCESSED_DIR / filename
    if filepath.exists():
        data, bounds = load_raster(filepath)
        im = ax.imshow(data, cmap=cmap)
        ax.set_title(title)
        plt.colorbar(im, ax=ax, shrink=0.7)
    else:
        ax.text(0.5, 0.5, 'Not found', ha='center', va='center', transform=ax.transAxes)
        ax.set_title(title + ' (missing)')
    ax.set_xticks([])
    ax.set_yticks([])

plt.tight_layout()
plt.show()

## 5. Find Optimal Locations

In [ ]:
# Find top habitat areas
if habitat_path.exists():
    with rasterio.open(habitat_path) as src:
        data = src.read(1)
        transform = src.transform
        
    # Find cells with score >= 0.8
    high_quality = data >= 0.8
    high_quality_count = np.sum(high_quality)
    
    print(f"High-quality habitat (score >= 0.8):")
    print(f"  Cells: {high_quality_count:,}")
    print(f"  Area: {high_quality_count * 30 * 30 / 1e6:.1f} km²")
    
    # Find centroid of high-quality areas
    if high_quality_count > 0:
        rows, cols = np.where(high_quality)
        
        # Convert to coordinates
        xs = transform[2] + cols * transform[0]
        ys = transform[5] + rows * transform[4]
        
        centroid_x = np.mean(xs)
        centroid_y = np.mean(ys)
        
        print(f"\nCentroid of high-quality areas (UTM 10N):")
        print(f"  Easting: {centroid_x:.0f}")
        print(f"  Northing: {centroid_y:.0f}")

## 6. Validation with iNaturalist Data

To validate the model, compare against known chanterelle observations from iNaturalist.

In [ ]:
# Placeholder for iNaturalist validation
# To use this, download Cantharellus formosus observations from iNaturalist
# and save as a CSV with columns: latitude, longitude

inaturalist_path = PROJECT_ROOT / 'data' / 'raw' / 'inaturalist_chanterelles.csv'

if inaturalist_path.exists():
    import pandas as pd
    from pyproj import Transformer
    
    obs = pd.read_csv(inaturalist_path)
    print(f"Loaded {len(obs)} iNaturalist observations")
    
    # Transform to UTM
    transformer = Transformer.from_crs('EPSG:4326', 'EPSG:32610')
    obs['x'], obs['y'] = transformer.transform(obs['latitude'], obs['longitude'])
    
    # Sample habitat scores at observation locations
    with rasterio.open(habitat_path) as src:
        obs['habitat_score'] = [list(src.sample([(x, y)]))[0][0] 
                                for x, y in zip(obs['x'], obs['y'])]
    
    # Summarize
    print(f"\nHabitat scores at observation locations:")
    print(f"  Mean: {obs['habitat_score'].mean():.3f}")
    print(f"  Median: {obs['habitat_score'].median():.3f}")
    print(f"  Min: {obs['habitat_score'].min():.3f}")
    print(f"  Max: {obs['habitat_score'].max():.3f}")
else:
    print("No iNaturalist data found for validation.")
    print(f"To validate, download Cantharellus formosus observations and save to:")
    print(f"  {inaturalist_path}")

## 7. Export Summary Statistics

In [ ]:
# Generate summary report
summary = {
    'habitat': {},
    'access': {},
}

if habitat_path.exists():
    data, _ = load_raster(habitat_path)
    valid = data.compressed()
    
    summary['habitat'] = {
        'min': float(valid.min()),
        'max': float(valid.max()),
        'mean': float(valid.mean()),
        'std': float(valid.std()),
        'pct_good': float(np.sum(valid >= 0.6) / len(valid) * 100),
        'pct_excellent': float(np.sum(valid >= 0.8) / len(valid) * 100),
    }

if access_path.exists():
    data, _ = load_raster(access_path)
    valid = data.compressed()
    
    summary['access'] = {
        'min': float(valid.min()),
        'max': float(valid.max()),
        'mean': float(valid.mean()),
        'std': float(valid.std()),
        'pct_good': float(np.sum(valid >= 0.6) / len(valid) * 100),
        'pct_excellent': float(np.sum(valid >= 0.8) / len(valid) * 100),
    }

# Display
import json
print(json.dumps(summary, indent=2))